In [14]:
import pandas as pd
import numpy as np

In [15]:
traffic = pd.read_csv("../dataset/last_data/accident_0_min_before.csv", encoding='cp949')
traffic_5min = pd.read_csv("../dataset/last_data/accident_5_min_before.csv", encoding='cp949')
traffic_10min = pd.read_csv("../dataset/last_data/accident_10_min_before.csv", encoding='cp949')
traffic_15min = pd.read_csv("../dataset/last_data/accident_15_min_before.csv", encoding='cp949')


non_traffic = pd.read_csv("../dataset/last_data/non_accident_0_min_before.csv", encoding='cp949')
non_traffic_5min = pd.read_csv("../dataset/last_data/non_accident_5_min_before.csv", encoding='cp949')
non_traffic_10min = pd.read_csv("../dataset/last_data/non_accident_10_min_before.csv", encoding='cp949')
non_traffic_15min = pd.read_csv("../dataset/last_data/non_accident_15_min_before.csv", encoding='cp949')

In [16]:
# non_traffic에서 랜덤으로 25136개 추출
non_traffic = non_traffic.sample(n=25136, random_state=42)
non_traffic_5min = non_traffic_5min.sample(n=25136, random_state=42)
non_traffic_10min = non_traffic_10min.sample(n=25136, random_state=42)
non_traffic_15min = non_traffic_15min.sample(n=25136, random_state=42)

In [17]:
static_traffic = traffic[['lanes', 'speed_limit', 'length', 'bump', 'camera',]]
static_non_traffic = non_traffic[['lanes', 'speed_limit', 'length', 'bump', 'camera',]]

In [18]:
drop_traffic_data = traffic.drop(['datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera' ,'label'], axis=1)
drop_non_traffic_data = non_traffic.drop([ 'datetime', 'link', 'lanes','speed_limit','length','bump', 'camera', 'label'], axis=1)

drop_traffic_data_5 = traffic_5min.drop(['datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)
drop_non_traffic_data_5 = non_traffic_5min.drop([ 'datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera','label'], axis=1)

drop_traffic_data_10 = traffic_10min.drop(['datetime', 'link', 'lanes','speed_limit','length' ,'bump', 'camera','label'], axis=1)
drop_non_traffic_data_10 = non_traffic_10min.drop([ 'datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)

drop_traffic_data_15 = traffic_15min.drop(['datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera','label'], axis=1)
drop_non_traffic_data_15 = non_traffic_15min.drop(['datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)

traffic_label = traffic['label']
non_traffic_label = non_traffic['label']

In [19]:
dynamic_traffic_5 = drop_traffic_data_5.values
dynamic_traffic_10 = drop_traffic_data_10.values
dynamic_traffic_15 = drop_traffic_data_15.values

dynamic_non_traffic_5 = drop_non_traffic_data_5.values
dynamic_non_traffic_10 = drop_non_traffic_data_10.values
dynamic_non_traffic_15 = drop_non_traffic_data_15.values

traffic_label = traffic_label.values
non_traffic_label = non_traffic_label.values

static_traffic = static_traffic.values
static_non_traffic = static_non_traffic.values

In [20]:
accident_data_5min = np.concatenate((dynamic_traffic_5, static_traffic), axis=1)
accident_data_10min = np.concatenate((dynamic_traffic_10, static_traffic), axis=1)
accident_data_15min = np.concatenate((dynamic_traffic_15, static_traffic), axis=1)

non_accident_data_5min = np.concatenate((dynamic_non_traffic_5, static_non_traffic), axis=1)
non_accident_data_10min = np.concatenate((dynamic_non_traffic_10, static_non_traffic), axis=1)
non_accident_data_15min = np.concatenate((dynamic_non_traffic_15, static_non_traffic), axis=1)

In [21]:
from sklearn.model_selection import train_test_split

X, Y = [], []

for i in range(len(accident_data_5min)):
    X.append([accident_data_15min[i], accident_data_10min[i], accident_data_5min[i]])
    Y.append(traffic_label[i])

for i in range(len(non_accident_data_5min)):
    X.append([non_accident_data_15min[i], non_accident_data_10min[i], non_accident_data_5min[i]])
    Y.append(non_traffic_label[i])


X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=40, shuffle=True)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=40, shuffle=True)

In [22]:
from keras.models import Model
from keras.layers import Dense, Input, Flatten
import tensorflow as tf


input_data = Input(shape=(X_train.shape[1], X_train.shape[2]))

x = Dense(100, activation='relu')(input_data)
x = Dense(100, activation='relu')(x)
x = Flatten()(x)
x = Dense(1, activation='sigmoid')(x)

result = Model(inputs=input_data, outputs=x)
result.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.BinaryAccuracy(threshold=0.5)])

In [23]:
hist = result.fit(X_train, Y_train, epochs=30, batch_size=32, validation_data=(X_val, Y_val))

loss, acc, auc, precision, recall, binary_accuracy = result.evaluate(X_test, Y_test, verbose=0)
print("m parameter", result.count_params())
print('loss : ', loss)
print('acc : ', acc)
print('auc : ', auc)
print('precision : ', precision)
print('recall : ', recall)
print('binary_accuracy : ', binary_accuracy)
#print f1-score with calculate recall precision

f1_score = (2 * precision * recall) / (precision + recall)
print('f1_score : ', f1_score)

Epoch 1/30
1257/1257 [==============================] - 1s 931us/step - loss: 0.5644 - accuracy: 0.6924 - auc_2: 0.7705 - precision_2: 0.7061 - recall_2: 0.6608 - binary_accuracy: 0.6924 - val_loss: 0.5396 - val_accuracy: 0.7161 - val_auc_2: 0.7967 - val_precision_2: 0.7538 - val_recall_2: 0.6447 - val_binary_accuracy: 0.7161
Epoch 2/30
1257/1257 [==============================] - 1s 982us/step - loss: 0.5374 - accuracy: 0.7185 - auc_2: 0.7983 - precision_2: 0.7439 - recall_2: 0.6677 - binary_accuracy: 0.7185 - val_loss: 0.5463 - val_accuracy: 0.7082 - val_auc_2: 0.8004 - val_precision_2: 0.6792 - val_recall_2: 0.7926 - val_binary_accuracy: 0.7082
Epoch 3/30
1257/1257 [==============================] - 1s 832us/step - loss: 0.5321 - accuracy: 0.7237 - auc_2: 0.8039 - precision_2: 0.7462 - recall_2: 0.6793 - binary_accuracy: 0.7237 - val_loss: 0.5369 - val_accuracy: 0.7193 - val_auc_2: 0.8064 - val_precision_2: 0.7916 - val_recall_2: 0.5979 - val_binary_accuracy: 0.7193
Epoch 4/30
1257/